### Imports and Dependencies

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import io
import requests
import os

from scipy.stats import zscore
from sklearn import preprocessing, metrics
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split
from sklearn.ensemble import ExtraTreesClassifier

import tensorflow as tf
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

from sklearn.preprocessing import LabelEncoder
from IPython.display import display

from imblearn.over_sampling import SMOTE

from matplotlib import pyplot
import matplotlib.pyplot as plot
import matplotlib.pyplot as plt

Panda Preferences

In [2]:
# Set options for displaying maximum columns and rows
pd.set_option('display.max_columns', None)  
#pd.set_option('display.max_rows', None)

**Read CSV**

In [3]:
# Read the CSV file into a DataFrame
df = pd.read_csv("../../../Datasets/Flows/iomt_flows.csv")

# Display the DataFrame
display(df)

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:3051: DtypeWarning: Columns (6,7,8) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id.orig_h,id.orig_p,id.resp_h,id.resp_p,proto,service,duration,orig_bytes,resp_bytes,conn_state,local_orig,local_resp,missed_bytes,history,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,tunnel_parents,flow_duration,fwd_pkts_tot,bwd_pkts_tot,fwd_data_pkts_tot,bwd_data_pkts_tot,fwd_pkts_per_sec,bwd_pkts_per_sec,flow_pkts_per_sec,down_up_ratio,fwd_header_size_tot,fwd_header_size_min,fwd_header_size_max,bwd_header_size_tot,bwd_header_size_min,bwd_header_size_max,flow_FIN_flag_count,flow_SYN_flag_count,flow_RST_flag_count,fwd_PSH_flag_count,bwd_PSH_flag_count,flow_ACK_flag_count,fwd_URG_flag_count,bwd_URG_flag_count,flow_CWR_flag_count,flow_ECE_flag_count,fwd_pkts_payload.min,fwd_pkts_payload.max,fwd_pkts_payload.tot,fwd_pkts_payload.avg,fwd_pkts_payload.std,bwd_pkts_payload.min,bwd_pkts_payload.max,bwd_pkts_payload.tot,bwd_pkts_payload.avg,bwd_pkts_payload.std,flow_pkts_payload.min,flow_pkts_payload.max,flow_pkts_payload.tot,flow_pkts_payload.avg,flow_pkts_payload.std,fwd_iat.min,fwd_iat.max,fwd_iat.tot,fwd_iat.avg,fwd_iat.std,bwd_iat.min,bwd_iat.max,bwd_iat.tot,bwd_iat.avg,bwd_iat.std,flow_iat.min,flow_iat.max,flow_iat.tot,flow_iat.avg,flow_iat.std,payload_bytes_per_second,fwd_subflow_pkts,bwd_subflow_pkts,fwd_subflow_bytes,bwd_subflow_bytes,fwd_bulk_bytes,bwd_bulk_bytes,fwd_bulk_packets,bwd_bulk_packets,fwd_bulk_rate,bwd_bulk_rate,active.min,active.max,active.tot,active.avg,active.std,idle.min,idle.max,idle.tot,idle.avg,idle.std,fwd_init_window_size,bwd_init_window_size,fwd_last_window_size,bwd_last_window_size,traffic,is_attack
0,10.10.10.252,56248,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,78,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,50.0,50.0,50.0,50.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,50.0,50.0,50.0,50.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,50.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
1,10.10.10.252,50214,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0,46.0,46.0,46.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,46.0,46.0,46.0,46.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
2,10.10.10.252,58369,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,73,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,45.0,45.0,45.0,45.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,45.0,45.0,45.0,45.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
3,10.10.10.252,57377,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,0,0,0,0,0,0,0,46.0,46.0,46.0,46.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,46.0,46.0,46.0,46.000000,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000,0.000000,1.0,0.0,46.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0,0,0,0,0,arpspoofing,1
4,10.10.10.252,12345,224.0.0.251,5353,udp,dns,-,-,-,S0,T,F,0,D,1,74,0,0,-,0.000000,1,0,1,0,0.000000,0.000000,0.000000,0.000000,8,8,8,0,0,0,0,0,0,

Fix Dataframe Mixed Types

In [4]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0     766915
1    2476273
Name: is_attack, dtype: int64

In [5]:
# Remove rows with '-' character in columns 6, 7 and 8
cols_to_check = ['duration', 'orig_bytes', 'resp_bytes']

# Create a boolean mask to identify rows with '-' character
mask = df[cols_to_check].apply(lambda x: x.str.contains('-', na=False)).any(axis=1)

# Use the mask to filter out rows with '-' character
df = df[~mask]

# Replace comma with period as decimal separator
cols_to_float = ['duration']

# Replace ',' with '.' in specified columns
df[cols_to_float] = df[cols_to_float].replace(',', '.', regex=True)

# Convert columns 7 and 8 to float and int data types
cols_to_int = ['orig_bytes', 'resp_bytes']

# Convert specified columns to float and int types
df[cols_to_float] = df[cols_to_float].astype(float)
df[cols_to_int] = df[cols_to_int].astype(int)

In [6]:
df.groupby('is_attack')['is_attack'].count()

is_attack
0    490249
1    756975
Name: is_attack, dtype: int64

-----------------------------------------------------------

## DF Statistics and Info

In [7]:
def display_information_dataframe(df_cop):
    # Create a summary of data types, column names, and unique values
    summary_data = [{'Data Type': dtype, 'Column Name': col, 'Unique Values': df_cop[col].unique()} for col, dtype in df_cop.dtypes.iteritems()]
    
    # Create a DataFrame from the summary data
    summary_df = pd.DataFrame(summary_data)
    
    # Set display options to show all rows and columns
    pd.options.display.max_rows = None
    pd.options.display.max_columns = None
    
    # Return the summary DataFrame
    return summary_df

In [8]:
display_information_dataframe(df)

,Data Type,Column Name,Unique Values
0,object,id.orig_h,"[10.10.10.252, 10.10.10.247, 10.10.10.248, 10...."
1,int64,id.orig_p,"[5353, 44007, 57669, 32939, 51580, 49728, 5762..."
2,object,id.resp_h,"[224.0.0.251, 10.10.10.249, 10.10.10.255, 172...."
3,int64,id.resp_p,"[5353, 22, 57621, 5683, 1883, 53, 67, 80, 5572..."
4,object,proto,"[udp, tcp, icmp]"
5,object,service,"[dns, -, mqtt, dhcp, http]"
6,float64,duration,"[1.999524, 1.162033, 0.196591, 3.001097, 0.195..."
7,int64,orig_bytes,"[90, 170, 579, 1608, 583, 580, 80, 88, 135, 40..."
8,int64,resp_bytes,"[0, 477, 94, 300, 1200, 509, 571, 400, 6, 493,..."
9,object,conn_state,"[S0, OTH, SHR, SF, SH, REJ, RSTR, S1, RSTOS0, ..."


--------------------------------------------

## Pre-processing and Data Encoding

Split History

In [9]:
def count_letters(string, is_upper):
    count = 0
    
    # Iterate through each character in the string
    for c in string:
        if is_upper and c.isupper():  # Check if the character is uppercase
            count += 1
        elif not is_upper and c.islower():  # Check if the character is lowercase
            count += 1
    
    return count

In [10]:
print('[SPLIT HISTORY] history_originator')
df['history_originator'] = df['history'].apply(lambda x: count_letters(x, True))

print('[SPLIT HISTORY] history_responder')
df['history_responder'] = df['history'].apply(lambda x: count_letters(x, False))

[SPLIT HISTORY] history_originator
[SPLIT HISTORY] history_responder


Decimal Scale Normalization

In [11]:
# Multiply by 1000 for a bigger scale
print('[DECIMAL SCALE NORMALIZATION] flow_duration')
df['flow_duration'] *= 1000

# Multiply by 100 for a bigger scale
print('[DECIMAL SCALE NORMALIZATION] duration')
df['duration'] *= 100

# Multiply by 10 for a bigger scale
print('[DECIMAL SCALE NORMALIZATION] down_up_ratio')
df['down_up_ratio'] *= 10

# Divide by 1000 for a smaller scale
print('[DECIMAL SCALE NORMALIZATION] down_up_ratio')
df['orig_bytes'] /= 1000

[DECIMAL SCALE NORMALIZATION] flow_duration
[DECIMAL SCALE NORMALIZATION] duration
[DECIMAL SCALE NORMALIZATION] down_up_ratio
[DECIMAL SCALE NORMALIZATION] down_up_ratio


Data Normalization

In [12]:
# Update values in 'bwd_PSH_flag_count' column if they are higher than 4
print('[DATA NORMALIZATION] bwd_PSH_flag_count')
df.loc[df['bwd_PSH_flag_count'] > 4, 'bwd_PSH_flag_count'] = 5

# Update values in 'bwd_data_pkts_tot' column if they are higher than 3
print('[DATA NORMALIZATION] bwd_data_pkts_tot')
df.loc[df['bwd_data_pkts_tot'] > 3, 'bwd_data_pkts_tot'] = 5

[DATA NORMALIZATION] bwd_PSH_flag_count
[DATA NORMALIZATION] bwd_data_pkts_tot


Remove Outliers

In [13]:
def remove_outliers(df,columns,n_std):
    for col in columns:
        print(f'[REMOVED OUTLIERS] {col}')
        
        mean = df[col].mean()
        sd = df[col].std()
        
        df = df[(df[col] <= mean+(n_std*sd))]
        
    return df

In [14]:
outliers = [
    'resp_ip_bytes',
    'orig_pkts',
    'flow_duration',
    'bwd_pkts_tot',
    'bwd_pkts_payload.avg',
    'bwd_header_size_tot',
    'resp_pkts',
    'duration',
]

df = remove_outliers(df, outliers, 3)

[REMOVED OUTLIERS] resp_ip_bytes
[REMOVED OUTLIERS] orig_pkts
[REMOVED OUTLIERS] flow_duration
[REMOVED OUTLIERS] bwd_pkts_tot
[REMOVED OUTLIERS] bwd_pkts_payload.avg
[REMOVED OUTLIERS] bwd_header_size_tot
[REMOVED OUTLIERS] resp_pkts
[REMOVED OUTLIERS] duration


One Hot Encoding

In [15]:
def one_hot_encoding(df, columns):
    for col in columns:
        print(f'[ONE HOT ENCODING] {col}')
        df = pd.get_dummies(df, columns=[col], prefix=col)  # Perform one-hot encoding on the column
    
    return df

In [16]:
cols_to_encode = [
    'proto',
    'local_resp',
    'local_orig',
    'fwd_header_size_min',
    'fwd_header_size_max',
    'flow_SYN_flag_count',
    'flow_RST_flag_count',
    'flow_FIN_flag_count',
    'conn_state',
    'bwd_PSH_flag_count',
    'bwd_header_size_min',
    'bwd_header_size_max',
    'resp_pkts',
]

df = one_hot_encoding(df,cols_to_encode)

[ONE HOT ENCODING] proto
[ONE HOT ENCODING] local_resp
[ONE HOT ENCODING] local_orig
[ONE HOT ENCODING] fwd_header_size_min
[ONE HOT ENCODING] fwd_header_size_max
[ONE HOT ENCODING] flow_SYN_flag_count
[ONE HOT ENCODING] flow_RST_flag_count
[ONE HOT ENCODING] flow_FIN_flag_count
[ONE HOT ENCODING] conn_state
[ONE HOT ENCODING] bwd_PSH_flag_count
[ONE HOT ENCODING] bwd_header_size_min
[ONE HOT ENCODING] bwd_header_size_max
[ONE HOT ENCODING] resp_pkts


Normalize, Z-score

In [17]:
def zscore_normalization(df, cols):
    # Standardize the selected columns
    for col in cols:
        if col not in df.columns:
            print(f"[WARNING] {col} not found in DataFrame.")
            continue
        print(f"[Z-SCORE] {col}")
        df[col] = zscore(df[col])
    
    print("[DONE] Z-score Normalization")
    print("[INFO] Current Fields in the DataFrame:")
    return df

In [18]:
cols_to_zscore = [
    'resp_ip_bytes',
    'payload_bytes_per_second',
    'orig_pkts',
    'orig_ip_bytes',
    'fwd_subflow_pkts',
    'fwd_subflow_bytes',
    'fwd_PSH_flag_count',
    'fwd_pkts_tot',
    'fwd_pkts_per_sec',
    'fwd_pkts_payload.tot',
    'fwd_pkts_payload.std',
    'fwd_pkts_payload.min',
    'fwd_pkts_payload.max',
    'fwd_pkts_payload.avg',
    'fwd_last_window_size',
    'fwd_init_window_size',
    'fwd_iat.tot',
    'fwd_iat.std',
    'fwd_iat.min',
    'fwd_iat.max', 
    'fwd_iat.avg',
    'fwd_header_size_tot',
    'fwd_data_pkts_tot',
    'fwd_bulk_bytes',
    'flow_pkts_per_sec',
    'flow_pkts_payload.tot',
    'flow_pkts_payload.std',
    'flow_pkts_payload.min',
    'flow_pkts_payload.max',
    'flow_pkts_payload.avg',
    'flow_iat.tot',
    'flow_iat.std',
    'flow_iat.min',
    'flow_iat.max',
    'flow_iat.avg',
    'flow_duration',
    'flow_ACK_flag_count',
    'down_up_ratio',
    'bwd_subflow_pkts',
    'bwd_subflow_bytes',
    'bwd_pkts_tot',
    'bwd_pkts_per_sec',
    'bwd_pkts_payload.tot',
    'bwd_pkts_payload.std',
    'bwd_pkts_payload.max',
    'bwd_pkts_payload.avg',
    'bwd_last_window_size',
    'bwd_init_window_size',
    'bwd_iat.tot',
    'bwd_iat.min',
    'bwd_iat.max',
    'bwd_iat.avg',
    'bwd_header_size_tot',
    'active.tot',
    'active.min',
    'active.max',
    'active.avg',
    'orig_bytes',
]

df = zscore_normalization(df, cols_to_zscore)

[Z-SCORE] resp_ip_bytes
[Z-SCORE] payload_bytes_per_second
[Z-SCORE] orig_pkts
[Z-SCORE] orig_ip_bytes
[Z-SCORE] fwd_subflow_pkts
[Z-SCORE] fwd_subflow_bytes
[Z-SCORE] fwd_PSH_flag_count
[Z-SCORE] fwd_pkts_tot
[Z-SCORE] fwd_pkts_per_sec
[Z-SCORE] fwd_pkts_payload.tot
[Z-SCORE] fwd_pkts_payload.std
[Z-SCORE] fwd_pkts_payload.min
[Z-SCORE] fwd_pkts_payload.max
[Z-SCORE] fwd_pkts_payload.avg
[Z-SCORE] fwd_last_window_size
[Z-SCORE] fwd_init_window_size
[Z-SCORE] fwd_iat.tot
[Z-SCORE] fwd_iat.std
[Z-SCORE] fwd_iat.min
[Z-SCORE] fwd_iat.max
[Z-SCORE] fwd_iat.avg
[Z-SCORE] fwd_header_size_tot
[Z-SCORE] fwd_data_pkts_tot
[Z-SCORE] fwd_bulk_bytes
[Z-SCORE] flow_pkts_per_sec
[Z-SCORE] flow_pkts_payload.tot
[Z-SCORE] flow_pkts_payload.std
[Z-SCORE] flow_pkts_payload.min
[Z-SCORE] flow_pkts_payload.max
[Z-SCORE] flow_pkts_payload.avg
[Z-SCORE] flow_iat.tot
[Z-SCORE] flow_iat.std
[Z-SCORE] flow_iat.min
[Z-SCORE] flow_iat.max
[Z-SCORE] flow_iat.avg
[Z-SCORE] flow_duration
[Z-SCORE] flow_ACK_flag_co

Delete Insignificant Columns from the Dataframe

In [19]:
def delete_columns(df, cols):
    for col in cols:
        df.drop(col, axis = 1, inplace = True)
        print(f'[REMOVED] {col}')
    
    return df

In [20]:
cols_to_del = [
    'tunnel_parents',
    'service',
    'missed_bytes',
    'idle.tot',
    'idle.std',
    'idle.min',
    'idle.max',
    'idle.avg',
    'fwd_URG_flag_count',
    'fwd_bulk_rate',
    'fwd_bulk_packets',
    'flow_ECE_flag_count',
    'flow_CWR_flag_count',
    'bwd_URG_flag_count',
    'bwd_pkts_payload.min',
    'bwd_iat.std',
    'bwd_bulk_rate',
    'bwd_bulk_packets',
    'bwd_bulk_bytes',
    'active.std',
    'id.orig_h',
    'id.orig_p',
    'id.resp_h',
    'id.resp_p',
    'history',
]

df = delete_columns(df,cols_to_del)

[REMOVED] tunnel_parents
[REMOVED] service
[REMOVED] missed_bytes
[REMOVED] idle.tot
[REMOVED] idle.std
[REMOVED] idle.min
[REMOVED] idle.max
[REMOVED] idle.avg
[REMOVED] fwd_URG_flag_count
[REMOVED] fwd_bulk_rate
[REMOVED] fwd_bulk_packets
[REMOVED] flow_ECE_flag_count
[REMOVED] flow_CWR_flag_count
[REMOVED] bwd_URG_flag_count
[REMOVED] bwd_pkts_payload.min
[REMOVED] bwd_iat.std
[REMOVED] bwd_bulk_rate
[REMOVED] bwd_bulk_packets
[REMOVED] bwd_bulk_bytes
[REMOVED] active.std
[REMOVED] id.orig_h
[REMOVED] id.orig_p
[REMOVED] id.resp_h
[REMOVED] id.resp_p
[REMOVED] history


---------------------------------------

## Create Model & Train Model

In [21]:
x_columns = df.columns.drop('is_attack')

le = LabelEncoder()
le.fit(df["is_attack"].values)

x = df[x_columns].values
y = df["is_attack"].values
y = le.transform(y)

In [22]:
# Split into training and testing sets
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=42, stratify=y)

In [23]:
x_test.shape,y_test.shape

((273370, 131), (273370,))

In [24]:
x_train.shape,y_train.shape

((820107, 131), (820107,))

---------------------------------------

## Save Train and Test Dataset

In [25]:
# Convert arrays to Series or DataFrame
x_train = pd.DataFrame(x_train, columns=x_columns)
y_train = pd.Series(y_train, name='is_attack')
x_test = pd.DataFrame(x_test, columns=x_columns)
y_test = pd.Series(y_test, name='is_attack')

# Create new DataFrames for training and testing sets
train_df = pd.concat([x_train, y_train], axis=1)
test_df = pd.concat([x_test, y_test], axis=1)

# Directory to save the files
save_dir = '../../../Datasets/Flows/'

# Save train and test DataFrames to CSV files in the specified directory
train_path = os.path.join(save_dir, 'train.csv')
test_path = os.path.join(save_dir, 'test.csv')

train_df.to_csv(train_path, index=False)
test_df.to_csv(test_path, index=False)
